# Задание 1. House Prices

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
plt.style.use('ggplot')

In [2]:
from sklearn import set_config

set_config(display='diagram')

## Загружаем данные

Это можно сделать вручную или через kaggle-api. Подготовительные шаги:
* Установить kaggle: `pip install kaggle`.
* Зайти в `Your profile` -> вкладка `Account` -> раздел `API`.
* Нажать `Create new API Token`, положить файл по нужному адресу (он будет в подсказке).

In [3]:
!kaggle competitions download -c house-prices-advanced-regression-techniques
!unzip -f house-prices-advanced-regression-techniques.zip
!rm -f house-prices-advanced-regression-techniques.zip

  0%|                                                | 0.00/199k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 199k/199k [00:00<00:00, 3.18MB/s]
Archive:  house-prices-advanced-regression-techniques.zip


Читаем данные:

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.SalePrice /= 100000 # делим для стабильных вычислений

train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,2.085
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,1.815
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,2.235
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,1.400
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,2.500


Признаки, по которым будем предсказывать:

In [5]:
num_features = [
    'YearBuilt',
    'GrLivArea',
    'TotRmsAbvGrd',
    'GarageArea',
]

Подготовка X и y:

In [6]:
X = train[num_features].copy()
y = train.SalePrice.copy()

X_test = test[num_features].fillna(0.0)

Нормируем данные:

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


transformer = ColumnTransformer([
    ('numerical', StandardScaler(), num_features),
])

Модель:

In [8]:
from sklearn.linear_model import LassoCV


lasso = LassoCV(precompute=True)

Пайплайн для правильного обучения:

In [9]:
model = Pipeline([
    ('transfomer', transformer),
    ('lasso', lasso)
])
model.fit(X, y)

Pipeline(steps=[('transfomer',
                 ColumnTransformer(transformers=[('numerical', StandardScaler(),
                                                  ['YearBuilt', 'GrLivArea',
                                                   'TotRmsAbvGrd',
                                                   'GarageArea'])])),
                ('lasso', LassoCV(precompute=True))])

Оценка качества на кроссвалидации:

In [10]:
from sklearn.metrics import mean_squared_error, make_scorer


def rmse(true, pred):
    return np.sqrt(mean_squared_error(true, pred))

rmse_score = make_scorer(rmse, greater_is_better=False)

In [11]:
from sklearn.model_selection import cross_val_score, KFold


cv = KFold(50, shuffle=True, random_state=42)
rmse_best = cross_val_score(model, X, y, cv=cv, n_jobs=-1, scoring=rmse_score, verbose=True)
np.mean(rmse_best)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    2.4s finished


-0.42701792967034286

### Отправка решения

In [12]:
preds = model.predict(X_test)

submission = pd.DataFrame({
    'Id': test.Id,
    'SalePrice': preds * 100000 # не забываем домножить на 100000
})
submission.to_csv('./baseline_submission.csv', index=False)

In [13]:
!cat ./baseline_submission.csv | head

Id,SalePrice
1461,142331.6493706337
1462,141985.26028093923
1463,213849.04087141226
1464,208738.3663632241
1465,183341.7587004845
1466,206844.4204330592
1467,165362.84386468763
1468,190093.34635616862
1469,187180.04309476036


Промежуточные сабмиты именовать по правилу: `MLDA, <ФИО>, <краткое описание модели>`.

У итогового сабмита добавить слово `финальный` в конце.

In [ ]:
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f baseline_submission.csv -m "<sumbit name>"